In [250]:
import pandas as pd

pd.set_option('mode.chained_assignment', None)

In [251]:
hc_relationship_df = pd.read_csv('hc_relationship_table.csv')

### Born

In [252]:
bio_details = hc_relationship_df[hc_relationship_df.i_header == "Biographical details"]

bio_details["i_data"] = bio_details.i_data.str.replace('\n',' ', regex=True)

bio_details_born = bio_details[bio_details.i_label == "Born"]
bio_details_born[["drop_col","num_date", "str_date", "age", "location"]] = bio_details_born[bio_details_born.i_label == "Born"].i_data.str.split('\(|\)', expand=True)
bio_details_born.drop('drop_col', axis=1, inplace=True)

bio_details_born = bio_details_born.iloc[:, -6:]
bio_details_born["age"] = bio_details_born.age.str.replace('age', '').str.strip()

bio_details_born['location'] = bio_details_born['location'].str.replace(r"\[.*?\]","", regex=True)

bio_details_born.to_csv('data/bio_details_born.csv')
bio_details_born.head()

,coach_name,id,num_date,str_date,age,location
3,Ryan Silverfield,Q80119196,1980-08-04,"August 4, 1980",43,
37,Eric Morris,Q5387136,1985-10-26,"October 26, 1985",37,"Littlefield, Texas, U.S."
51,Mike Bloomgren,Q6846062,1977-01-25,"January 25, 1977",46,"Tallahassee, Florida, U.S."
66,Rhett Lashlee,Q7320465,1983-06-09,"June 9, 1983",40,"Springdale, Arkansas, U.S."
80,Alex Golesh,Q105496742,1984-06-24,"June 24, 1984",39,"Moscow, Russian SFSR, Soviet Union"


### Coaches csv

In [253]:
df_born = pd.read_csv('data/bio_details_born.csv')
df_born.drop(df_born.columns[0], axis=1, inplace=True)

df_born.to_csv("data/coaches.csv", index=False)
df_born.head()

,coach_name,id,num_date,str_date,age,location
0,Ryan Silverfield,Q80119196,1980-08-04,"August 4, 1980",43,NaN
1,Eric Morris,Q5387136,1985-10-26,"October 26, 1985",37,"Littlefield, Texas, U.S."
2,Mike Bloomgren,Q6846062,1977-01-25,"January 25, 1977",46,"Tallahassee, Florida, U.S."
3,Rhett Lashlee,Q7320465,1983-06-09,"June 9, 1983",40,"Springdale, Arkansas, U.S."
4,Alex Golesh,Q105496742,1984-06-24,"June 24, 1984",39,"Moscow, Russian SFSR, Soviet Union"


### Coach At Relationship

In [254]:
df_cc = pd.read_csv("coach_career.csv", converters={'coach_role': pd.eval, 'year': pd.eval})

df_cc.rename(columns={"org": "school", "id":"coach_id", "link":"school_id"}, inplace=True)
df_cc = df_cc[['title', 'school_id', 'coach_name', 'coach_id', 'school', 'coach_role', 'year']]

ex_school_id_list = ['/wiki/Graduate_assistant', '/wiki/Offensive_coordinator', '/wiki/Defensive_coordinator']
df_cc = df_cc[~df_cc.school_id.isin(ex_school_id_list)]
df_cc.head()


,title,school_id,coach_name,coach_id,school,coach_role,year
0,The Bolles School,/wiki/The_Bolles_School,Ryan Silverfield,Q80119196,Bolles School,[GA],[1999]
2,Hampden–Sydney Tigers football,/wiki/Hampden%E2%80%93Sydney_Tigers_football,Ryan Silverfield,Q80119196,Hampden–Sydney,[off. asst.],[2000]
3,NaN,NaN,Ryan Silverfield,Q80119196,Hampden–Sydney,[DL],"[2001, 2002]"
4,NaN,NaN,Ryan Silverfield,Q80119196,Hampden–Sydney,[TE],[2003]
5,NaN,NaN,Ryan Silverfield,Q80119196,Memorial Day HS,[HC],[2004]


In [255]:
# Check for Null links
print("check for null links...\n")
df_cc[df_cc['school_id'].isnull()]

check for null links...



,title,school_id,coach_name,coach_id,school,coach_role,year
3,NaN,NaN,Ryan Silverfield,Q80119196,Hampden–Sydney,[DL],"[2001, 2002]"
4,NaN,NaN,Ryan Silverfield,Q80119196,Hampden–Sydney,[TE],[2003]
5,NaN,NaN,Ryan Silverfield,Q80119196,Memorial Day HS,[HC],[2004]
9,NaN,NaN,Ryan Silverfield,Q80119196,Minnesota Vikings,[DL],"[2009, 2010]"
10,NaN,NaN,Ryan Silverfield,Q80119196,Minnesota Vikings,[OL],"[2011, 2013]"
...,...,...,...,...,...,...,...
1313,NaN,NaN,Will Hall,Q8002843,West Alabama,[HC],"[2011, 2013]"
1324,NaN,NaN,Jon Sumrall,Q109679821,Tulane,"[co-DC, LB]",[2014]
1327,NaN,NaN,Jon Sumrall,Q109679821,Kentucky,[ILB],"[2019, 2020]"
1328,NaN,NaN,Jon Sumrall,Q109679821,Kentucky,"[co-DC, ILB]",[2021]


In [256]:
# Self backfill
df_cc_gb = df_cc.groupby(["school"])["school_id"].first().reset_index()
df_cc_null = df_cc[df_cc['school_id'].isnull()]

df_cc_backfill = df_cc_null.merge(df_cc_gb, on="school", suffixes=("_", ""))

df_cc_not_null = df_cc[df_cc['school_id'].notnull()]
df_cc_merged = pd.concat([df_cc_not_null, df_cc_backfill], ignore_index=True)

df_cc_merged.drop("school_id_", axis=1, inplace=True)
df_cc_merged.head()


,title,school_id,coach_name,coach_id,school,coach_role,year
0,The Bolles School,/wiki/The_Bolles_School,Ryan Silverfield,Q80119196,Bolles School,[GA],[1999]
1,Hampden–Sydney Tigers football,/wiki/Hampden%E2%80%93Sydney_Tigers_football,Ryan Silverfield,Q80119196,Hampden–Sydney,[off. asst.],[2000]
2,Jacksonville Dolphins football,/wiki/Jacksonville_Dolphins_football,Ryan Silverfield,Q80119196,Jacksonville,[QB],[2005]
3,UCF Knights football,/wiki/UCF_Knights_football,Ryan Silverfield,Q80119196,UCF,[GA],"[2006, 2007]"
4,Minnesota Vikings,/wiki/Minnesota_Vikings,Ryan Silverfield,Q80119196,Minnesota Vikings,[OQC],[2008]


In [257]:
# Missing IDs
print("check for null links...\n")
df_cc_merged[df_cc_merged['school_id'].isnull()]

check for null links...



,title,school_id,coach_name,coach_id,school,coach_role,year
852,NaN,None,Ryan Silverfield,Q80119196,Memorial Day HS,[HC],[2004]
896,NaN,None,Willie Fritz,Q8021517,Willis HS,[assistant],[1986]
959,NaN,None,Pat Narduzzi,Q7143830,Rhode Island,[LB],"[1993, 1997]"
981,NaN,None,Kalani Sitake,Q6350585,BYU,[HC],"[2016, present]"
1102,NaN,None,Chris Creighton,Q5106302,Limhamn Griffins,[HC],[1993]
1115,NaN,None,Tim Albin,Q63342501,Northwestern Oklahoma State,"[OC, OL]","[1994, 1996]"
1116,NaN,None,Tim Albin,Q63342501,Northwestern Oklahoma State,[HC],"[1997, 1999]"
1168,NaN,None,Butch Jones,Q5002395,Ferris State,[RB],[1995]
1169,NaN,None,Butch Jones,Q5002395,Ferris State,[OC],"[1996, 1997]"


In [258]:
# Write to CSV
df_cc_merged.to_csv("data/coach_at.csv", index=False)

### School stuff

In [259]:
# Unique schools id's from the "Coach At" relationship
unq_school_list = list(df_cc_merged.school_id.str.strip().unique())
unq_school_df = df_cc[["school", "school_id"]].drop_duplicates(subset=['school'])
# unq_school_df = df_cc[["school", "school_id"]]

unq_school_df.head()

,school,school_id
0,Bolles School,/wiki/The_Bolles_School
2,Hampden–Sydney,/wiki/Hampden%E2%80%93Sydney_Tigers_football
5,Memorial Day HS,NaN
6,Jacksonville,/wiki/Jacksonville_Dolphins_football
7,UCF,/wiki/UCF_Knights_football


In [260]:
unq_school_df[unq_school_df.school == "BYU"]

,school,school_id
308,BYU,NaN


In [261]:
# D1 schools from school-coach table
df_d1_school = pd.read_csv("school_coach_table.csv")

df_d1_school = df_d1_school[["School", "Conference", "team_url"]]
df_d1_school.rename(columns={"School": "school", "Conference":"conference", "team_url":"id"}, inplace=True)

df_d1_school["type"] = "D1"

df_d1_school.head()

,school,conference,id,type
0,Florida Atlantic Owls,The American,/wiki/Florida_Atlantic_Owls_football,D1
1,Memphis Tigers,The American,/wiki/Memphis_Tigers_football,D1
2,Navy Midshipmen,The American,/wiki/Navy_Midshipmen_football,D1
3,North Texas Mean Green,The American,/wiki/North_Texas_Mean_Green_football,D1
4,Rice Owls,The American,/wiki/Rice_Owls_football,D1


In [265]:
# df_d1_school[df_d1_school.school=="BYU Cougars"]
# unq_school_df[unq_school_df.school=="BYU"]

In [267]:
# Merge to get confrenece and alt_school data from d1-school-coach table
df_school = unq_school_df.merge(df_d1_school, left_on="school_id", right_on="id", how="left")
df_school.rename(columns={"school_x": "school", "school_y":"alt_school"}, inplace=True)

df_school.drop_duplicates(subset=["school"], keep='first', inplace=True)
print("check for null links...\n", df_school['school'].is_unique)
df_school

check for null links...
 True


,school,school_id,alt_school,conference,id,type
0,Bolles School,/wiki/The_Bolles_School,NaN,NaN,NaN,NaN
1,Hampden–Sydney,/wiki/Hampden%E2%80%93Sydney_Tigers_football,NaN,NaN,NaN,NaN
2,Memorial Day HS,NaN,NaN,NaN,NaN,NaN
3,Jacksonville,/wiki/Jacksonville_Dolphins_football,NaN,NaN,NaN,NaN
4,UCF,/wiki/UCF_Knights_football,UCF Knights,Big 12,/wiki/UCF_Knights_football,D1
...,...,...,...,...,...,...
334,Henderson State,/wiki/Henderson_State_Reddies_football,NaN,NaN,NaN,NaN
335,Southwest Baptist,/wiki/Southwest_Baptist_Bearcats_football,NaN,NaN,NaN,NaN
336,Arkansas–Monticello,/wiki/Arkansas%E2%80%93Monticello_Boll_Weevils...,NaN,NaN,NaN,NaN
337,West Alabama,/wiki/West_Alabama_Tigers_football,NaN,NaN,NaN,NaN


In [268]:
# Non D1 Nulls
df_non_d1 = df_school[df_school.type != "D1"]
df_non_d1 = df_non_d1[df_non_d1['school_id'].isnull()]
df_non_d1

,school,school_id,alt_school,conference,id,type
2,Memorial Day HS,NaN,NaN,NaN,NaN,NaN
60,Willis HS,NaN,NaN,NaN,NaN,NaN
115,Rhode Island,NaN,NaN,NaN,NaN,NaN
143,BYU,NaN,NaN,NaN,NaN,NaN
173,Kentucky,NaN,NaN,NaN,NaN,NaN
255,Limhamn Griffins,NaN,NaN,NaN,NaN,NaN
265,Northwestern Oklahoma State,NaN,NaN,NaN,NaN,NaN
314,Ferris State,NaN,NaN,NaN,NaN,NaN


In [269]:
# D1 Nulls
df_d1 = df_school[df_school.type == "D1"]
df_d1 = df_d1[df_d1['school_id'].isnull()]

df_d1

,school,school_id,alt_school,conference,id,type


In [270]:
# Back fill null "school_id" with "school"
df_school['school_id'] = df_school['school_id'].fillna(df_school['school'])

print(df_school[df_school['school_id'].isnull()])

df_school.to_csv("data/schools.csv", index=False)

df_school

Empty DataFrame
Columns: [school, school_id, alt_school, conference, id, type]
Index: []


,school,school_id,alt_school,conference,id,type
0,Bolles School,/wiki/The_Bolles_School,NaN,NaN,NaN,NaN
1,Hampden–Sydney,/wiki/Hampden%E2%80%93Sydney_Tigers_football,NaN,NaN,NaN,NaN
2,Memorial Day HS,Memorial Day HS,NaN,NaN,NaN,NaN
3,Jacksonville,/wiki/Jacksonville_Dolphins_football,NaN,NaN,NaN,NaN
4,UCF,/wiki/UCF_Knights_football,UCF Knights,Big 12,/wiki/UCF_Knights_football,D1
...,...,...,...,...,...,...
334,Henderson State,/wiki/Henderson_State_Reddies_football,NaN,NaN,NaN,NaN
335,Southwest Baptist,/wiki/Southwest_Baptist_Bearcats_football,NaN,NaN,NaN,NaN
336,Arkansas–Monticello,/wiki/Arkansas%E2%80%93Monticello_Boll_Weevils...,NaN,NaN,NaN,NaN
337,West Alabama,/wiki/West_Alabama_Tigers_football,NaN,NaN,NaN,NaN


In [231]:
# test_df = df_d1_school.merge(df_school[df_school.type == "D1"], left_on="id", right_on="school_id", how="left")


# test_df = test_df[test_df['school_id_y'].isnull()]
# # print(test_df)
# test_df

### School csv starts here

In [34]:
# Integrating 
alt_name_df = hc_relationship_df[["title", "link"]]
alt_name_df.head()

,title,link
0,Cincinnati,/wiki/Cincinnati
1,Simi Valley High School,/wiki/Simi_Valley_High_School
2,Cal Lutheran Kingsmen and Regals,/wiki/Cal_Lutheran_Kingsmen_and_Regals#Football
3,Hampden–Sydney College,/wiki/Hampden%E2%80%93Sydney_College
4,The Bolles School,/wiki/The_Bolles_School


In [35]:
df_school_alt = df_school.merge(alt_name_df, left_on="id", right_on="link", how="left")
print(df_school_alt.shape)

df_school_alt_check = df_school_alt[df_school_alt['title'].isnull()]
print(df_school_alt_check)

df_school_alt = df_school_alt[df_school_alt['title'].notnull()]
print(df_school_alt.shape)


(659, 5)
                  school conference                                  id title  \
656  Texas State Bobcats   Sun Belt  /wiki/Texas_State_Bobcats_football   NaN   

    link  
656  NaN  
(658, 5)


In [47]:
df_school_alt = df_school_alt[df_school_alt['title'].notnull()]
df_school_alt.drop_duplicates(inplace=True)

print("check that id are unique: ", len(df_school_alt) - len(df_school_alt.id.unique()))
print(df_school_alt.head())



check that id are unique:  0
                    school    conference  \
0    Florida Atlantic Owls  The American   
1           Memphis Tigers  The American   
13         Navy Midshipmen  The American   
16  North Texas Mean Green  The American   
18               Rice Owls  The American   

                                       id                            title  \
0    /wiki/Florida_Atlantic_Owls_football   Florida Atlantic Owls football   
1           /wiki/Memphis_Tigers_football          Memphis Tigers football   
13         /wiki/Navy_Midshipmen_football         Navy Midshipmen football   
16  /wiki/North_Texas_Mean_Green_football  North Texas Mean Green football   
18               /wiki/Rice_Owls_football               Rice Owls football   

                                     link  
0    /wiki/Florida_Atlantic_Owls_football  
1           /wiki/Memphis_Tigers_football  
13         /wiki/Navy_Midshipmen_football  
16  /wiki/North_Texas_Mean_Green_football  
18               